<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Итоговый-вывод:" data-toc-modified-id="Итоговый-вывод:-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Итоговый вывод:</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

- **Таблица insurance**
- `Пол` - 1-муж, 0-жен
- `Возраст` - 
- `Зарплата` - ₽
- `Члены семьи` - Кол-во
- `Страховые выплаты` - Кол-во
- `Целевой признак`: количество страховых выплат клиенту за последние 5 лет

In [1]:
# загужаем библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
from numpy.random import RandomState

## Загрузка данных

In [2]:
#загружаем данные
data = pd.read_csv('insuranc.csv')

In [3]:
#первые 5 строк
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [4]:
#размер таблицы
data.shape

(5000, 5)

In [5]:
#описательная статистика
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


In [6]:
#смотрим минимальную зарплату
data.loc[data['Зарплата'] == data['Зарплата'].min()]

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
726,1,20.0,5300.0,1,0


In [7]:
#смотрим максимальную зарплату
data.loc[data['Зарплата'] == data['Зарплата'].max()]

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
3255,1,41.0,79000.0,2,0


In [8]:
#больше всех получил страховых выплат
data.loc[data['Страховые выплаты'] == data['Страховые выплаты'].max()]

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
228,1,65.0,39700.0,1,5


In [9]:
#смотрим типы данных
data.dtypes

Пол                    int64
Возраст              float64
Зарплата             float64
Члены семьи            int64
Страховые выплаты      int64
dtype: object

In [10]:
#считаем пропуски
data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

In [11]:
#Посмотрим на количество дубликатов в таблице
data.duplicated() == True

0       False
1       False
2       False
3       False
4       False
        ...  
4995    False
4996    False
4997    False
4998    False
4999    False
Length: 5000, dtype: bool

In [12]:
data[data.duplicated()]

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
281,1,39.0,48100.0,1,0
488,1,24.0,32900.0,1,0
513,0,31.0,37400.0,2,0
718,1,22.0,32600.0,1,0
785,0,20.0,35800.0,0,0
...,...,...,...,...,...
4793,1,24.0,37800.0,0,0
4902,1,35.0,38700.0,1,0
4935,1,19.0,32700.0,0,0
4945,1,21.0,45800.0,0,0


**Вывод:**
- Познакомились с таблицей: 5000 объектов и 5 признаков для каждого
- Аномалий не обнаружено
- Пропусков не обнаружено

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Обоснование:**
- Задача предсказания матрица признаков умноженная на обратимую матрицу умноженная на вектор весов
$$
a' = XPw
$$

- Задача обучения сводится к 
$$
w' = \arg\min_w MSE(XPw, y)
$$

- Формула обучения 
$$
w' = ((XP)^T XP)^{-1} (XP)^T y
$$
- Раскроем скобки

$$
w' = (P^TX^TXP)^{-1} P^TX^T y
$$

$$
w' = P^{-1} (X^TX)^{-1} (P^T)^{-1} P^TX^T y
$$

$$
(P^T)^{-1} P^T = E =>
$$

$$
w' = P^{-1}(X^TX)^{-1} EX^T y, EA = AE = A => = P^{-1} (X^TX)^{-1} X^T y
$$
- Тогда формула обучения это
$$
w' = P^{-1} (X^TX)^{-1} X^T y
$$
- Тогда формула предсказания
$$
a' = XPP^{-1}w =>
$$

$$
PP^{-1} = E =>
$$

$$
a' = XEw
$$
- Так как AE = EA = A, то
$$
a' = XEw = Xw = a
$$

**Ответ:** 
- Можно заметить что в формуле обучения добавился новый коэффициент $P^{-1}$, но если подставить этот коэффициент в формулу предсказаний, то предсказания с добавлением нового коэффициента и без не изменятся
- Изменятся только веса признаков

In [13]:
#создаем признаки и целевой
X = data.drop('Страховые выплаты',axis=1)
y = data['Страховые выплаты']

In [14]:
#создаем и обучаем модель
model = LinearRegression()
model.fit(X, y)

LinearRegression()

In [15]:
#делаем предсказания
predictions = model.predict(X)

In [16]:
#считаем целевую метрику
print(r2_score(y, predictions))

0.42494550286668


## Алгоритм преобразования

In [17]:
#зафиксируем случайность для воспроизводимости
np.random.seed(0)
matrix = np.random.rand(4,4)

In [18]:
matrix

array([[0.5488135 , 0.71518937, 0.60276338, 0.54488318],
       [0.4236548 , 0.64589411, 0.43758721, 0.891773  ],
       [0.96366276, 0.38344152, 0.79172504, 0.52889492],
       [0.56804456, 0.92559664, 0.07103606, 0.0871293 ]])

In [19]:
a = np.eye(4)
b = np.eye(4)

In [20]:
# E @ E = E
a @ b

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [21]:
#произведение любой матрицы и еденичной = любой матрице
a @ matrix

array([[0.5488135 , 0.71518937, 0.60276338, 0.54488318],
       [0.4236548 , 0.64589411, 0.43758721, 0.891773  ],
       [0.96366276, 0.38344152, 0.79172504, 0.52889492],
       [0.56804456, 0.92559664, 0.07103606, 0.0871293 ]])

In [22]:
#проверим обратимость матрицы
np.linalg.inv(matrix)

array([[-2.93661054,  0.62452847,  1.79269745,  1.09060329],
       [ 1.66428382, -0.43068264, -1.077603  ,  0.54137516],
       [ 3.929769  , -1.95008776, -0.51994879, -1.46025687],
       [-1.73862477,  2.09349729,  0.18396553, -0.19368226]])

In [23]:
#преобразуем признаки
X_matrix = X @ matrix

In [24]:
#обучим модель с учетом новых признаков
model_matrix = LinearRegression()
model_matrix.fit(X_matrix, y)

LinearRegression()

In [25]:
#сделаем предсказание
predictions_matrix = model_matrix.predict(X_matrix)

In [26]:
#посмотрим разницу
print('На преобразованных данных:', r2_score(y, predictions_matrix))
print('На обычных данных:', r2_score(y, predictions))

На преобразованных данных: 0.42494550286668886
На обычных данных: 0.42494550286668


In [27]:
#Расчитаем веса w для исходных признаков по формуле w = (X^T X)^{-1} X^T y
w = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

#Расчитаем вектор предсказаний по исходным признакам по формуле 𝑎 = 𝑋𝑤
a = X @ w

#Преобразованная матрица признаков
features_m = X_matrix

#Расчитаем веса  𝑤1  для преобразованных признаков по формуле  w = (X^T X)^{-1} X^T y
w1 = np.linalg.inv(features_m.T.dot(features_m)).dot(features_m.T).dot(y)

#Расчитаем вектор предсказаний по преобразованным признакам по формуле  𝑎 = 𝑋𝑤
a1 = features_m @ w1

#Получим вектор разниц между предсказаниями по исходным признакам  𝑎  и преобразованным  𝑎1 :
interval = a - a1

#Считаем разницу предсказаний
interval.sum()

-0.005209977673066003

**Обоснование**
- Разница минимальна

## Проверка алгоритма

## Итоговый вывод:
- Познакомились с таблицей, удалили дубликаты
- Предложили алгоритм защиты данных, на математически агрументировали смысл этого преобразования

- Данный способ можно использовать как метод защиты данных, т.к предсказание модели на обычных признаках и на преобразованных отличаются на уровне погрешности
- Качество модели не изменится от преобразования признаков, изменятся только их веса и модель придется переобучать заново

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования